Этот код соберет в одном месте следующие функции для обработки текста на русском языке в формате класса:
чистка от пунктуации,
лемматизация,
стемматизация,
удаление стоп-слов,
поиск самых частотных н-грам,
сентенизация,
токенизация,
выделение и нормализация именованных сущностей,
возвращение формата UD,
анализ тональности, 
и вывод общей статистики по тексту.


In [ ]:
#все импорты
import spacy
nlp=spacy.load("ru_core_news_sm")
nlp.add_pipe('sentencizer')

import re
from pathlib import Path
from collections import Counter

import nltk
nltk.download('stopwords')
from nltk.text import Text
from nltk.tokenize import word_tokenize
from nltk.corpus import stopwords
from nltk.stem.snowball import SnowballStemmer
from nltk.util import ngrams

from transformers import pipeline





[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\erokh\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [48]:
class Token:
    def __init__(self, idx, form, lemma, pos, feats, head, deprel):
        self.idx = idx
        self.form = form
        self.lemma = lemma
        self.pos = pos
        self.feats = feats
        self.head = head
        self.deprel = deprel

In [91]:
class Text_ru: 
    def __init__(self, data, name):
        self.data=data
        self.name=name

    def write(self, spec, done): #записать в соответствующий файл то, что сделали
        with open(self.name+'_'+spec+'.txt', 'w', encoding='UTF-8') as f:
            f.write(str(done))
    
    def clean(self): #очистить от пунктуации и переносов строк
        data=self.data
        data=re.sub(r'[^\w\s]','',data)
        data=re.sub(r'\n', '', data)
        return data
    
    def lemmatize(self): #лемматизировать с помощью функции токенизировать внизу, лемматизация от спейси не очень хорошая, но не хочется использовать слишком много библиотек
        lemmas=[]
        tokens=self.tokenize()
        for t in tokens:
            if t.pos != 'PUNCT':
                lemmas.append(t.lemma)
        return lemmas
        
    def stemmatize(self): #стемматизировать с помощью nltk
        stemmer=SnowballStemmer('russian')  
        stems=[]
        data=self.clean()
        tokens=nltk.word_tokenize(data)
        for token in tokens:
            stems.append(stemmer.stem(token))
        return stems

    def delete_stopwords(self): #удалить стопслова с помощью nltk и вернуть текст для дальнейшего использования
        data=self.data
        stop_words=set(stopwords.words('russian'))
        tokens=word_tokenize(data)
        filtered=[w for w in tokens if not w.lower() in stop_words]
        return ' '.join(filtered)
    
    def find_common_ngrams(self, n=3, common=10):
        data=self.clean()
        tokens=nltk.word_tokenize(data)
        text_nltk=Text(tokens)
        c=Counter(list(ngrams(text_nltk, n)))
        return c.most_common(common)

    def sentenize(self): #сентенизировать с помощью spacy
        data=self.data
        doc=nlp(data)
        sents=[sent.text for sent in doc.sents]
        return sents  

    def tokenize(self): #токенизировать с помощью spacy и создать объекты класса Token для удобного использования
        tokens=[]
        data=self.data
        doc=nlp(data)
        for sent in doc.sents:
            for token in sent:
                if token.dep_=='ROOT':
                    head=0
                else:
                    head=token.head.i-sent.start+1
                id=token.i - sent.start+1
                t=Token(id, token.text, token.lemma_, token.pos_, str(token.morph), head, token.dep_,)
                tokens.append(t)
        return tokens

    def extract_ent(self): #выделить именнованные сущности
        ents=[]
        data=self.data
        doc=nlp(data)
        for ent in doc.ents:
            ents.append([ent.text, ent.label_])
        return ents
    
    def to_UD(self): #напечатать в формате UD
        tokens=self.tokenize()
        for t in tokens:
            print(t.idx, 
                t.form, 
                t.lemma, 
                t.pos, 
                '_', 
                t.feats, 
                t.head,
                t.deprel,
                '_',
                '_', sep='\t')
    
    def sent_analyze(self): #модель не может анализировать слишком большой текст, поэтому делаем это попредложенно.
        sentiment_pipeline = pipeline('sentiment-analysis', model='blanchefort/rubert-base-cased-sentiment')
        results=[]
        sents=self.sentenize()
        for sent in sents:
            result = sentiment_pipeline(sent)
            results.append(result)
        return results
    
    def stats(self):
        #токены и предложения
        aver = round(len(self.tokenize())/len(self.sentenize()))
        unique = round(len(set(self.tokenize()))/len(self.tokenize())*100)
        print(f'Кол-во токенов {len(self.tokenize())}\nКол-во предложений {len(self.sentenize())}\nСредняя длина предложения {aver}\n')

        #статистика по pos
        poses = set()
        for t in self.tokenize():
            poses.add(t.pos)
        poses = list(poses)

        for i in range(len(poses)):
            quant_i = 0
            for t in self.tokenize():
                if t.pos == poses[i]:
                    quant_i+=1
            print(f'{poses[i]}: {quant_i} токен(а/ов)')
        
        #леммы и стемы
        c_lemms=Counter(self.lemmatize())
        c_stems=Counter(self.stemmatize())
        print(f'\nКол-во лемм: {len(self.lemmatize())}\nКол-во уникальных лемм: {len(c_lemms)}\n10 самымх частотных лем: {c_lemms.most_common(10)}')
        print(f'Кол-во стем: {len(self.stemmatize())}\nКол-во уникальных стем: {len(c_stems)}\n10 самымх частотных стем: {c_stems.most_common(10)}\n')

        #н-граммы
        print(f'Самые частотные биграммы: {self.find_common_ngrams(2, 5)}\nСамые частотные триграммы: {self.find_common_ngrams(3, 5)}\n')

        #стоп слова
        data=self.data
        stop_words=set(stopwords.words('russian'))
        tokens=word_tokenize(data)
        sw_text=[w for w in tokens if w.lower() in stop_words]
        c_sw=Counter(sw_text)
        print(f'В тексте можно удалить {len(sw_text)} стопслов(а).\nСамые частотные из них {c_sw.most_common(5)}\n')

        #именнованные сущности
        tags=[]
        names=[]
        ents=self.extract_ent()
        for i in range(len(ents)):
            tags.append(ents[i][1])
            names.append(ents[i][0])
        c_tags=Counter(tags)
        c_names=Counter(names)
        print(f'Кол-во именнованных сущностей: {len(ents)}\nРаспределения по тегам: PER - {c_tags["PER"]}, LOC - {c_tags["LOC"]}\nСамые частотные именнованные сущности {c_names.most_common(5)}\n')

        #анализ тональности:
        labels=[]
        result=self.sent_analyze()
        for i in range(len(result)):
            labels.append(result[i][0]['label'])
        c_labels=Counter(labels)
        print(f'Кол-во позитивных предложений - {c_labels["POSITIVE"]}, нейтральных - {c_labels["NEUTRAL"]} и негативных - {c_labels["NEGATIVE"]}.')


In [92]:
with open('project_ru.txt', 'r', encoding='UTF-8') as f:
    data=f.read()
metel=Text_ru(data, 'project_ru')

'''metel.clean()
metel.stemmatize()
metel.tokenize()
metel.sentenize()
metel.extract_ent()
metel.delete_stopwords()
metel.find_common_ngrams()
metel.extract_ent()
metel.to_UD()
metel.sent_analyze()'''


'metel.clean()\nmetel.stemmatize()\nmetel.tokenize()\nmetel.sentenize()\nmetel.extract_ent()\nmetel.delete_stopwords()\nmetel.find_common_ngrams()\nmetel.extract_ent()\nmetel.to_UD()\nmetel.sent_analyze()'

In [93]:
metel.stats()

Кол-во токенов 4402
Кол-во предложений 267
Средняя длина предложения 16

ADP: 328 токен(а/ов)
ADV: 225 токен(а/ов)
ADJ: 264 токен(а/ов)
PUNCT: 900 токен(а/ов)
PRON: 359 токен(а/ов)
PROPN: 178 токен(а/ов)
DET: 112 токен(а/ов)
AUX: 86 токен(а/ов)
NUM: 38 токен(а/ов)
SCONJ: 72 токен(а/ов)
X: 9 токен(а/ов)
PART: 122 токен(а/ов)
NOUN: 787 токен(а/ов)
VERB: 668 токен(а/ов)
CCONJ: 190 токен(а/ов)
SPACE: 64 токен(а/ов)

Кол-во лемм: 3502
Кол-во уникальных лемм: 1369
10 самымх частотных лем: [('и', 147), ('в', 110), ('не', 68), ('что', 59), ('\n', 58), ('она', 56), ('он', 52), ('быть', 51), ('с', 44), ('гаврилович', 32)]
Кол-во стем: 3355
Кол-во уникальных стем: 1272
10 самымх частотных стем: [('и', 146), ('в', 101), ('он', 93), ('не', 80), ('был', 76), ('что', 54), ('е', 49), ('с', 44), ('на', 31), ('владимир', 30)]

Самые частотные биграммы: [(('Марья', 'Гавриловна'), 17), (('и', 'с'), 6), (('Гаврила', 'Гаврилович'), 5), (('и', 'что'), 4), (('Марьи', 'Гавриловны'), 4)]
Самые частотные триграм